In [7]:
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf
from model import PnlpMixerSeqCls, PnlpMixerTokenCls
from mixer import FFFTrainFixed
from dataset import PnlpMixerDataModule
from run import PnlpMixerSeqClsTrainModule, PnlpMixerTokenClsTrainModule
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import time
import torch.nn as nn

In [11]:
cfg = OmegaConf.load('cfg/imdb_base.yml')

def get_module_cls(type: str): 
    if type == 'mtop': 
        return PnlpMixerTokenClsTrainModule
    if type == 'matis' or type == 'imdb': 
        return PnlpMixerSeqClsTrainModule

module_cls = get_module_cls(cfg.train.dataset_type)

data_module = PnlpMixerDataModule(cfg.vocab, cfg.train, cfg.model.projection)
data_module.setup('test')

module = module_cls(
    optimizer_cfg=cfg.train.optimizer,
    model_cfg=cfg.model
)

module.load_state_dict(torch.load("ffft_256_4_cpp.ckpt"))

module.eval()

test_dataloader = data_module.test_dataloader()

batch = next(iter(test_dataloader))

device = next(module.parameters()).device
batch = {k: v.to(device) for k, v in batch.items()}

{'num_mixers': 2, 'max_seq_len': 1024, 'hidden_dim': 256, 'mlp_hidden_dim': 256}


RuntimeError: Error(s) in loading state_dict for PnlpMixerSeqClsTrainModule:
	Missing key(s) in state_dict: "model.pnlp_mixer.mixer.mixers.0.mlp_1.linear_out.weight", "model.pnlp_mixer.mixer.mixers.0.mlp_2.linear_out.weight", "model.pnlp_mixer.mixer.mixers.1.mlp_1.linear_out.weight", "model.pnlp_mixer.mixer.mixers.1.mlp_2.linear_out.weight". 

In [ ]:
with torch.no_grad():
    start = time.time()
    logits = train_module.model(batch['inputs'])
    end = time.time()
    predictions = torch.argmax(logits, dim=1)

for i, (pred, target) in enumerate(zip(predictions, batch['targets'])):
    predicted_label = cfg.train.labels[pred.item()]
    true_label = cfg.train.labels[target.item()]
    # print(f"Sample {i}:")
    # print(f"  Predicted: {predicted_label}")
    # print(f"  Actual: {true_label}")
    # print()

accuracy = (predictions == batch['targets']).float().mean()
print(f"Batch Accuracy: {accuracy.item():.4f}")
print(f"Time: {end-start}")